In [ ]:
#configuration
import os
import sys

DATA_DIR = "../hydro_dash/data/studies"
# adding project dirs to path so code may be referenced from the notebook
sys.path.insert(0, '..')
sys.path.insert(0, '../evaluation/')
sys.path.insert(0, '../evaluation/queries/')

In [ ]:
from evaluation import utils, config
import queries2 # need to fix path to use original queries
#import dask_geopandas
import duckdb as ddb
#import spatialpandas as sp
#import hvplot.pandas # noqa


In [ ]:
%%time

from evaluation import utils, config
import queries2 # need to fix path to use original queries
import dask_geopandas

def get_all_catchment_metrics():
    basins_gdf = utils.parquet_to_gdf(config.HUC10_PARQUET_FILEPATH)
    query = queries2.calculate_catchment_metrics(
        config.MEDIUM_RANGE_FORCING_PARQUET,
        config.FORCING_ANALYSIS_ASSIM_PARQUET,
        group_by=["reference_time, catchment_id"],
        order_by=["reference_time, catchment_id"],
        filters=[
            {
                "column": "1",
                "operator": "=",
                "value": 1
            },
#            {
#                "column": "catchment_id",
#                "operator": "like",
#                "value": "" + huc2.value + "%"
#            },
#            {
#                "column": "reference_time",
#                "operator": "=",
#                "value": "2023-01-01 18:00:00"
#            }            
       ]
    )
    df = ddb.query(query).to_df()
    gdf_map = basins_gdf.merge(df, left_on="huc10", right_on="catchment_id")
    return gdf_map


dask_df = dask_geopandas.from_geopandas(get_all_catchment_metrics(), npartitions=32)
dask_df.head()

In [ ]:
%%time
dask_df['huc2'] = dask_df['huc10'].str[:2]
#print(gdf.dtypes)
dask_df['huc2'] = dask_df['huc2'].astype("int")
dask_df['huc10'] = dask_df['huc10'].astype("string")
dask_df['name'] = dask_df['name'].astype("string")
dask_df['reference_time'] = dask_df['reference_time'].astype("datetime64[ns]")
dask_df['catchment_id'] = dask_df['catchment_id'].astype("string")
dask_df['time'] = dask_df['reference_time'].astype(int)/ 10**9
dask_df['time'] = dask_df['time'].astype("int") # needs the extra pass to actually maintain int type
dask_df['huc2'] = dask_df['huc2'].astype("int") # needs the extra pass to actually maintain int type

dask_df.dtypes

In [ ]:
#centroid = dask_df[["geometry"]].dissolve().centroid
#print(centroid.x, centroid.y)
tst = dask_df['reference_time'].unique().compute()


In [ ]:
tst = dask_df[(dask_df['huc2']==1) & (dask_df['time'] == 1672531200)]
tst[['geometry', 'bias']].hvplot.polygons()


In [ ]:
import holoviews as hv, geoviews as gv, param, dask.dataframe as dd, cartopy.crs as crs
import panel as pn
from datetime import datetime as dt
from bokeh.models import HoverTool
#import datetime as dt

from colorcet import cm
from holoviews.operation.datashader import rasterize, shade, regrid
from holoviews.streams import RangeXY

hv.extension('bokeh', logo=False)
pn.extension(loading_spinner='dots', loading_color='#00aa41', sizing_mode="stretch_width")

opts = dict(width=900,
            height=600,
            xaxis=None,
            yaxis=None,
            bgcolor='black',
            show_grid=False)
cmaps = ['fire','bgy','bgyw','bmy','gray','kbc']


class HydroExplorer(param.Parameterized):
    alpha      = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    cmap       = param.ObjectSelector(cm['bgyw'], objects={c:cm[c] for c in cmaps})
    _max_time  = int(dask_df["time"].max().compute())
    _min_time  = int(dask_df["time"].min().compute())
    time       = param.Integer(_min_time, bounds=(_min_time, _max_time))
    #time       = param.ObjectSelector(default=int(dask_df["time"].min().compute()), objects=dask_df['time'].unique().compute())
    #time       = param.Date(dask_df["reference_time"].min().compute(),
    #                                     bounds=(dask_df["reference_time"].min().compute(), dask_df["reference_time"].max().compute()))
    #time       = param.ObjectSelector(default=dask_df["reference_time"].min().compute(), objects=sorted(dask_df["reference_time"].unique().compute()))
    #time       = pn.widgets.DateSlider(name='reference_time', start=dask_df["reference_time"].min().compute(), end=dask_df["reference_time"].max().compute())
    huc2       = param.ObjectSelector(default=1, objects=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])
    #huc2       = param.Integer(default=int(dask_df['huc2'].min().compute()), bounds=(int(dask_df['huc2'].min().compute()), int(dask_df['huc2'].max().compute())))
    #reference_time_max = dask_df['reference_time'].max().compute()
    #reference_time_min = dask_df['reference_time'].min().compute()
    #reference_time = datetime_range_slider = pn.widgets.DatetimeRangeSlider(
    #                                                                        name='reference_time',
    #                                                                        start=dt.datetime(reference_time_min.year, reference_time_min.month, reference_time_min.day), 
    #                                                                        end=dt.datetime(reference_time_max.year, reference_time_max.month, reference_time_max.day),
    #                                                                        value=(dt.datetime(reference_time_min.year, reference_time_min.month, reference_time_min.day), dt.datetime(reference_time_max.year, reference_time_max.month, reference_time_max.day)),
    #                                                                        step=10000
    #                                                                    )

    render_rasterized = True
    
    @param.depends('huc2', 'time')
    def polygons(self):
        rslt_df = dask_df[(dask_df['huc2']==self.huc2) & (dask_df['time']==self.time)]
        #rslt_df = dask_df[(dask_df['huc2']==self.huc2) & (dask_df['reference_time'] == '' + str(self.time) + '')]
        #rslt_df = rslt_df.to_crs("EPSG:3395")
        hover = HoverTool(tooltips=[('Name', '@name'),('Bias', '@bias')])
        return gv.Polygons(rslt_df.compute())


    def view(self,**kwargs):
        self.render_rasterized = True
        polygons = hv.DynamicMap(self.polygons).opts(tools=['hover'], toolbar='above')
        tiles = gv.tile_sources.StamenTerrain().apply.opts(alpha=self.param.alpha, **opts)
        if self.render_rasterized == False:
            final_map = tiles * polygons
        else:
            agg = rasterize(polygons, width=600, height=400, precompute=True).opts(hv.opts.Points(alpha=0.1, hover_alpha=0.2, size=10))
            final_map = (tiles * shade(agg, cmap=self.param.cmap))
        return final_map

hydro = HydroExplorer(name="data explorer")
pn.Column(hydro.param, hydro.view()).servable()


In [ ]:
import holoviews as hv, geoviews as gv, param, dask.dataframe as dd, cartopy.crs as crs
import panel as pn
from datetime import datetime as dt
from bokeh.models import HoverTool

from colorcet import cm
from holoviews.operation.datashader import rasterize, shade, regrid, datashade
from holoviews.streams import Stream, param
from holoviews import streams
from holoviews.plotting.links import DataLink

hv.extension('bokeh', logo=False)
pn.extension(loading_spinner='dots', loading_color='#00aa41', sizing_mode="stretch_width")

opts = dict(width=900,
            height=600,
            xaxis=None,
            yaxis=None,
            bgcolor='black',
            show_grid=False)
cmaps = ['fire','bgy','bgyw','bmy','gray','kbc']
huc2 = 1
name = "test"
time = 1672531200

class HydroExplorer(param.Parameterized):
    alpha      = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    cmap       = param.ObjectSelector(cm['bgyw'], objects={c:cm[c] for c in cmaps})
    time       = param.Integer(int(dask_df["time"].min().compute()), bounds=(int(dask_df['time'].min().compute()), int(dask_df['time'].max().compute())))
    #time       = param.Date(dt.fromtimestamp(dask_df["reference_time"].min().compute()),
    #                                     bounds=(dt.fromtimestamp(dask_df["reference_time"].min().compute()), dt.fromtimestamp(dask_df["reference_time"].max().compute())))
    #time       = pn.widgets.DateSlider(name='reference_time', start=dask_df["reference_time"].min().compute(), end=dask_df["reference_time"].max().compute())
    huc2       = param.ObjectSelector(default=1, objects=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])
    #huc2       = param.Integer(default=int(dask_df['huc2'].min().compute()), bounds=(int(dask_df['huc2'].min().compute()), int(dask_df['huc2'].max().compute())))
    render_rasterized = True
    
    @param.depends('huc2', 'time')
    def polygons(self):
        rslt_df = dask_df[(dask_df['huc2']==self.huc2) & (dask_df['time']==self.time)]
        #rslt_df = rslt_df.to_crs("EPSG:3395")
        tooltips = [
            ('Name', '@name'),
            ('Bias', '@bias')
        ]
        hover = HoverTool(tooltips=tooltips)
        points = gv.Polygons(rslt_df.compute()).opts(tools=[hover], toolbar='above')#, vdims=['name', 'bias'])
        
        if rslt_df['geometry'].count().compute() < 50: 
            self.render_rasterized = False
        else:
            self.render_rasterized = True

        #if self.hour != (0, 24): points = points.select(dropoff_hour=self.hour)
        return points
    

    def view(self,**kwargs):
        self.render_rasterized = False
        polygons = hv.DynamicMap(self.polygons)#.opts(tools=[hover], toolbar='above')
        rslt_df = dask_df[(dask_df['huc2']==self.huc2) & (dask_df['time']==self.time)]
        table = hv.Table(rslt_df[['name', 'bias']].compute(), [('name', 'Name'), ('bias', 'Bias')])
        DataLink(polygons, table)
        tiles = gv.tile_sources.StamenTerrain().apply.opts(alpha=self.param.alpha, **opts)
        if self.render_rasterized == False:
            final_map = (tiles * polygons) + table
        else:
            tooltips = [
                ('Name', '@name'),
                ('Bias', '@bias'),
                ('Test', 'What!')
            ]
            hover = HoverTool(tooltips=tooltips)
            agg = rasterize(polygons, width=600, height=400, precompute=True).opts(hv.opts.Points(tools=[hover], alpha=0.1, hover_alpha=0.2, size=10))
            final_map = ((tiles * shade(agg, cmap=self.param.cmap)) + table).cols(1)
        return final_map

hydro = HydroExplorer(name="data explorer")
pn.Row(hydro.param, hydro.view()).servable()


In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade
import numpy as np

points = hv.Points(np.random.multivariate_normal((0,0), [[0.1, 0.1], [0.1, 1.0]], (500000,)))


def filter_points(points, x_range, y_range):
    if x_range is None or y_range is None:
        return points
    return points[x_range, y_range]

def hover_points(points, threshold=5000):
    if len(points) > threshold:
        return points.iloc[:0]
    return points

range_stream = hv.streams.RangeXY(source=points)
streams=[range_stream]

threshold = pn.widgets.IntSlider(name='Threshold', start=1000, end=10000, step=100)
filtered = points.apply(filter_points, streams=streams)
hover = filtered.apply(hover_points, threshold=threshold)
shaded = datashade(filtered, width=400, height=400, streams=streams)

dynamic_hover = (shaded * hover).opts(
    hv.opts.Points(tools=['hover'], alpha=0.1, hover_alpha=0.2, size=10))

pn.Column(threshold, dynamic_hover)

In [ ]:
hv.DynamicMap?